In [ ]:
!pip install -q -U langchain transformers bitsandbytes accelerate gradio

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 975.5/975.5 kB 10.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.3/9.3 MB 27.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.8/119.8 MB 5.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 309.4/309.4 kB 15.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.3/12.3 MB 36.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 332.8/332.8 kB 19.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.5/127.5 kB 2.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.0/92.0 kB 5.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.2/318.2 kB 12.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 145.0/145.0 kB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.1/10.1 MB 67

In [ ]:
!pip install langchain-community langchain-core

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 8.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.2/49.2 kB 6.5 MB/s eta 0:00:00


import libraries

In [ ]:
import torch
from transformers import BitsAndBytesConfig
from langchain import PromptTemplate, LLMChain
from transformers import AutoModelForCausalLM, AutoTokenizer, StoppingCriteria, StoppingCriteriaList, pipeline

Quantization config

In [ ]:
quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True,
)

get user hf token key (modify if using colab secret)

*   this should be created on your HuggingFace account, in the settings
*   can be modified by using secrets on your colab



In [ ]:
from huggingface_hub import notebook_login
notebook_login()

4bits model and tokenizer

In [ ]:
model = AutoModelForCausalLM.from_pretrained( "mistralai/Mistral-7B-Instruct-v0.3", device_map="auto",quantization_config=quantization_config, )
tokenizer = AutoTokenizer.from_pretrained("mistralai/Mistral-7B-Instruct-v0.3")

In [ ]:
stop_list = ['\nHuman:', '\n```\n']

stop_token_ids = [tokenizer(x)['input_ids'] for x in stop_list]
stop_token_ids

In [ ]:
from torch import cuda
device = f'cuda:{cuda.current_device()}' if cuda.is_available() else 'cpu'
stop_token_ids = [torch.LongTensor(x).to(device) for x in stop_token_ids]
stop_token_ids

In [ ]:
class StopOnTokens(StoppingCriteria):
    def __call__(self, input_ids: torch.LongTensor, scores: torch.FloatTensor, **kwargs) -> bool:
        for stop_ids in stop_token_ids:
            if torch.eq(input_ids[0][-len(stop_ids):], stop_ids).all():
                return True
        return False

stopping_criteria = StoppingCriteriaList([StopOnTokens()])

In [ ]:
generate_text = pipeline(
    model=model,
    tokenizer=tokenizer,
    return_full_text=True,
    task='text-generation',
    stopping_criteria=stopping_criteria,
    temperature=0.7,  #  for more creative generation
    max_new_tokens=1500,
    # min_new_tokens=1000,
    repetition_penalty=1.2  # reduce repetition
)

In [ ]:
import gradio as gr
from langchain.prompts import PromptTemplate
from langchain.llms import HuggingFacePipeline

# Define the LLMs
llm_first = HuggingFacePipeline(pipeline=generate_text)
llm_second = HuggingFacePipeline(pipeline=generate_text)
llm_third = HuggingFacePipeline(pipeline=generate_text)
llm_fourth = HuggingFacePipeline(pipeline=generate_text)

# Define the prompt templates for each LLM
template_first = """
The Lester Dent Pulp Paper Master Fiction Plot Formula is a writing guide created by Lester Dent, a prolific pulp fiction writer best known for his work on the "Doc Savage" series. Dent's formula provides a structured approach to crafting engaging and action-packed stories. Here is a brief overview of the formula:

Introduction:
Introduce the hero and the central problem or conflict.
Set up a situation that hooks the reader's interest immediately.
First Quarter:
The hero tries to solve the problem.
Introduce complications and obstacles that thwart the hero's initial attempts.
Introduce other key characters (both allies and antagonists).
End with a twist or a surprising development.

Characters:
- Hero: {hero}
- Villain: {villain}
- Sidekick: {sidekick}
- Victim: {victim}
- Witness: {witness}

You are a story generator that follows the Lester Dent Pulp Paper formula.
Based on this formula, your task is to generate exactly the first 1500 words of the story in one continuous block of text. The text should be coherent, engaging, and flow like a real story without any interruptions or section headers.
"""

template_second = """
The Lester Dent Pulp Paper Master Fiction Plot Formula continues as follows:

Second Quarter:
Escalate the conflict and introduce additional problems for the hero.
The hero faces greater challenges and setbacks.
Include physical conflicts or action scenes.
Introduce a major plot twist or revelation that complicates the hero's mission.

This is the first part of the story:
{first_part}

Based on this formula, your task is to generate the next 1500 words of the story in one continuous block of text. The text should be coherent, engaging, and flow like a real story without any interruptions or section headers.
"""

template_third = """
The Lester Dent Pulp Paper Master Fiction Plot Formula continues as follows:

Third Quarter:
The hero makes some progress towards solving the problem but faces significant adversity.
Introduce new conflicts and obstacles.
The hero encounters the main villain or a critical turning point in the story.
End with a twist that puts the hero in an even worse situation.

This is the first part of the story:
{first_part}

This is the second part of the story:
{second_part}

Based on this formula, your task is to generate the next 1500 words of the story in one continuous block of text. The text should be coherent, engaging, and flow like a real story without any interruptions or section headers.
"""

template_fourth = """
The Lester Dent Pulp Paper Master Fiction Plot Formula continues as follows:

Final Quarter:
The hero faces the greatest challenges and is pushed to their limits.
All mysteries and plot threads are resolved.
The hero uses their skills, intelligence, and bravery to overcome the final obstacles.
End with a final twist or surprise that concludes the story in a satisfying way.

This is the first part of the story:
{first_part}

This is the second part of the story:
{second_part}

This is the third part of the story:
{third_part}

Based on this formula, your task is to generate the final 1500 words of the story which is the last quarter of the story so you should end the story in this chapter. The generated text should be in one continuous block of text. The text should be coherent, engaging, and flow like a real story without any interruptions or section headers.
"""

# Function to combine all parts into one text
def combine_story_parts(*parts):
    return "\n\n".join(parts)

# Function to generate the first part
def generate_first_part(hero, villain, sidekick, victim, witness):
    prompt_first = PromptTemplate.from_template(template_first)
    first_result = llm_first(prompt_first.format(hero=hero, villain=villain, sidekick=sidekick, victim=victim, witness=witness))
    return first_result

# Function to generate the second part
def generate_second_part(first_part):
    prompt_second = PromptTemplate.from_template(template_second)
    second_result = llm_second(prompt_second.format(first_part=first_part))
    return second_result

# Function to generate the third part
def generate_third_part(first_part, second_part):
    prompt_third = PromptTemplate.from_template(template_third)
    third_result = llm_third(prompt_third.format(first_part=first_part, second_part=second_part))
    return third_result

# Function to generate the final part
def generate_final_part(first_part, second_part, third_part):
    prompt_fourth = PromptTemplate.from_template(template_fourth)
    fourth_result = llm_fourth(prompt_fourth.format(first_part=first_part, second_part=second_part, third_part=third_part))
    return fourth_result

# Function to combine all parts and generate the full story
def generate_full_story(hero, villain, sidekick, victim, witness, first_part, second_part, third_part):
    # Generate the final part
    final_part = generate_final_part(first_part, second_part, third_part)

    # Combine all parts
    full_story = combine_story_parts(first_part, second_part, third_part, final_part)
    return full_story

# Define the Gradio interface
with gr.Blocks() as interface:
    gr.Markdown("# Story Generator")
    gr.Markdown("Generate a complete story based on the Lester Dent Pulp Paper formula.")

    with gr.Row():
        hero_input = gr.Textbox(label="Hero")
        villain_input = gr.Textbox(label="Villain")
        sidekick_input = gr.Textbox(label="Sidekick")
        victim_input = gr.Textbox(label="Victim")
        witness_input = gr.Textbox(label="Witness")
        generate_first = gr.Button("Generate First Part")

    first_part_output = gr.Textbox(label="First Part", lines=10)

    with gr.Row():
        generate_second = gr.Button("Generate Second Part")
        first_part_modified = gr.Textbox(label="Modified First Part", lines=10)

    second_part_output = gr.Textbox(label="Second Part", lines=10)

    with gr.Row():
        generate_third = gr.Button("Generate Third Part")
        second_part_modified = gr.Textbox(label="Modified Second Part", lines=10)

    third_part_output = gr.Textbox(label="Third Part", lines=10)

    with gr.Row():
        generate_final = gr.Button("Generate Final Part")
        third_part_modified = gr.Textbox(label="Modified Third Part", lines=10)

    final_part_output = gr.Textbox(label="Final Part", lines=10)

    generate_full = gr.Button("Generate Full Story")
    full_story_output = gr.Textbox(label="Full Story", lines=20)

    # Set up the interactions
    generate_first.click(
        fn=generate_first_part,
        inputs=[hero_input, villain_input, sidekick_input, victim_input, witness_input],
        outputs=[first_part_output]
    )

    generate_second.click(
        fn=generate_second_part,
        inputs=[first_part_output],
        outputs=[second_part_output]
    )

    generate_third.click(
        fn=generate_third_part,
        inputs=[first_part_modified, second_part_output],
        outputs=[third_part_output]
    )

    generate_final.click(
        fn=generate_final_part,
        inputs=[first_part_modified, second_part_modified, third_part_modified],
        outputs=[final_part_output]
    )

    generate_full.click(
        fn=generate_full_story,
        inputs=[hero_input, villain_input, sidekick_input, victim_input, witness_input, first_part_modified, second_part_modified, third_part_modified],
        outputs=[full_story_output]
    )

# Launch the Gradio interface
interface.launch()

Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://89193b4f405d4c4d8f.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
